# 0 TorchText

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
upoaded=files.upload()

Saving tweets.csv to tweets.csv


In [ ]:
df=pd.read_csv('tweets.csv')
df.shape

(1364, 2)

In [ ]:
df.labels.value_counts()

0    931
1    352
2     81
Name: labels, dtype: int64

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [ ]:
# Import Library
import random
import torch, torchtext
from torchtext.legacy import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

Here while definign the Tweet, below, inlcude_lengths is kept False is its not needed. We are not creating a padded sequence

In [ ]:
Tweet = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=False)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [ ]:
fields = [('tweets', Tweet),('labels',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [ ]:
example = [data.Example.fromlist([df.tweets[i],df.labels[i]], fields) for i in range(df.shape[0])] 

In [ ]:
# Creating dataset
twitterDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [ ]:
(train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [ ]:
(len(train), len(valid))

(1159, 205)

An example from the dataset:

In [ ]:
train.examples[10].tweets

['Obama',
 ',',
 'Romney',
 'agree',
 ':',
 'Admit',
 'women',
 'to',
 'Augusta',
 'golf',
 'club',
 ':',
 'US',
 'President',
 'Barack',
 'Obama',
 'believes',
 'women',
 'should',
 'be',
 'allowe',
 '...',
 'http://t.co/PVKrepqI']

In [ ]:
vars(train.examples[10])

{'labels': 0,
 'tweets': ['Obama',
  ',',
  'Romney',
  'agree',
  ':',
  'Admit',
  'women',
  'to',
  'Augusta',
  'golf',
  'club',
  ':',
  'US',
  'President',
  'Barack',
  'Obama',
  'believes',
  'women',
  'should',
  'be',
  'allowe',
  '...',
  'http://t.co/PVKrepqI']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [ ]:
Tweet.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [ ]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  4651
Size of label vocab :  3
Top 10 words appreared repeatedly : [('Obama', 1069), (':', 783), ('#', 780), ('.', 761), (',', 598), ('"', 550), ('the', 542), ('RT', 516), ('?', 419), ('to', 400)]
Labels :  defaultdict(None, {0: 0, 1: 1, 2: 2})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [ ]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and RNN modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into an RNN layer with 100 hidden features (again, we’re compressing down from the 300-dimensional ).
3. Finally, the output of the RNN (the final hidden state after processing the incoming tweet) is pushed through another RNN which takes the output of previous RNN and its own state and outputs a single vectore which is then pushed to  standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers=1):
        
        super().__init__()          
        self.hidden_dim=hidden_dim
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
              
        # RNN layer for encoding
        self.encoder = nn.RNN(embedding_dim,hidden_dim,batch_first=True)
        # RNN layer for decoding
        self.decoder=nn.RNN(hidden_dim,hidden_dim,batch_first=True)
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        
        # text = [batchsize, sent_length]
        # embedded = [batchsize, sent_len, embedding_dim]
        embedded = self.embedding(text)

       
        #Initialize the first hidden state
        hidden=torch.zeros(1,text.size(0),self.hidden_dim).to(device)

        output1, hidden = self.encoder(embedded,hidden)

        #hidden = [batchsize, 1,hidden_dim] as number of layers taken here is 1
        #The first hidden state which goes to encoder is the last single vector from the encoder. 
        #The output1 of encoder is of shape [batchsize, sent_len, hiddenn_dim], contains hidden states from each time_step 
        # and that goes as the input to the decoder RNN along with the final single vector from encoder (final hidden state)

        hidden1=hidden #final hidden vector from encoder RNN goes to first cell of decoder along with first output of encoder.
        output2,hidden1=self.decoder(output1,hidden1)

        #The output2 of decoder is of shape [batchsize, sent_len, hiddenn_dim], contains hidden states from each time_step 
        #hidden1 contains the last output and is sent to fully connected layer to make the final prediction.
        dense_outputs = self.fc(hidden1)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
        
        #Since we want to output at each time step of encoder and decoder, return output1 and output2 as well along with prediction(output)
            
        return output,output1,output2

In [ ]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 3
num_layers = 1


# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers)

In [ ]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(4651, 300)
  (encoder): RNN(300, 100, batch_first=True)
  (decoder): RNN(100, 100, batch_first=True)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
The model has 1,456,003 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [ ]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

##Trials to understand the shapes of the output
Next few cells can be ignored as they were the trials to understand the shape and outputs of the model to be able to capture what is expected from the assignment.

In [ ]:
b=next(iter(train_iterator))
t,l=b
l.size,size

In [ ]:
emb=nn.Embedding(size_of_vocab,embedding_dim).to(device)
#tt=t[0] as include length is switched off this is not needed.
te=emb(t)
te.size()

In [ ]:
pred,o1,o2,h,h1=model(t)

In [ ]:
o1.size()

torch.Size([32, 20, 100])

In [ ]:
o2.size()

torch.Size([32, 20, 100])

In [ ]:
h.size()

torch.Size([1, 32, 100])

In [ ]:
h1.size()

torch.Size([1, 32, 100])

In [ ]:
assert torch.equal(o1[:,-1,:],h1.squeeze(0))

In [ ]:
loss=criterion(pred, b.labels)

In [ ]:
loss.item()

1.0855475664138794

In [ ]:
loss.backward()
optimizer.step()      

This training loop is same taken from the notebook developed in the class on tweets data set, with suitable modifications to capture the hidden state at every time step.

**Training Loop**

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text
        tweet = batch.tweets   
        
        #Capture predictions and output of encode and decoder at every time step.
        #Although it is not needed while training it is captured to maintain consistency across.

        predictions,o1,o2= model(tweet)
        
        # compute the loss
        loss = criterion(predictions, batch.labels)   

        #As the model is overfitting to the training data, regularisation is introduced.     
        L2_lambda=0.001
        L2_norm=sum(p.pow(2.0).sum() for p in model.parameters())    
        loss=loss+L2_lambda*L2_norm    

        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [ ]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet = batch.tweets
            
            predictions,o1,o2= model(tweet)
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            L2_lambda=0.001
            L2_norm=sum(p.pow(2.0).sum() for p in model.parameters())    
            loss=loss+L2_lambda*L2_norm    
        
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [ ]:
N_EPOCHS = 50
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_valid_acc= valid_acc
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')
print(f'\t best Val. Loss: {best_valid_loss:.3f} |  best Val. Acc: {best_valid_acc*100:.2f}% \n')

	Train Loss: 1390.183 | Train Acc: 57.00%
	 Val. Loss: 1381.698 |  Val. Acc: 64.73% 

	Train Loss: 1373.775 | Train Acc: 69.88%
	 Val. Loss: 1365.462 |  Val. Acc: 67.86% 

	Train Loss: 1357.645 | Train Acc: 71.40%
	 Val. Loss: 1349.476 |  Val. Acc: 68.30% 

	Train Loss: 1341.753 | Train Acc: 72.08%
	 Val. Loss: 1333.711 |  Val. Acc: 68.75% 

	Train Loss: 1326.076 | Train Acc: 73.43%
	 Val. Loss: 1318.152 |  Val. Acc: 68.75% 

	Train Loss: 1310.613 | Train Acc: 73.52%
	 Val. Loss: 1302.789 |  Val. Acc: 68.75% 

	Train Loss: 1295.334 | Train Acc: 75.37%
	 Val. Loss: 1287.622 |  Val. Acc: 68.75% 

	Train Loss: 1280.247 | Train Acc: 77.99%
	 Val. Loss: 1272.641 |  Val. Acc: 68.75% 

	Train Loss: 1265.356 | Train Acc: 79.68%
	 Val. Loss: 1257.849 |  Val. Acc: 72.32% 

	Train Loss: 1250.653 | Train Acc: 80.69%
	 Val. Loss: 1243.248 |  Val. Acc: 72.32% 

	Train Loss: 1236.140 | Train Acc: 81.71%
	 Val. Loss: 1228.833 |  Val. Acc: 72.32% 

	Train Loss: 1221.807 | Train Acc: 82.47%
	 Val. Loss:

## Model Testing

In [ ]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    #tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tweet]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction,o1,o2= model(tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
import random
for i in range(10):
  r_n=random.choice(train)
  tweet=r_n.tweets
  #print(r_n)
  p=classify_tweet(tweet)
  print(f'{tweet} | sentiment : {p}')

['@KatrinaNation', 'Katrina', 'I', 'just', 'seen', 'the', 'HBO', 'documentary', 'hot', 'coffee', '.', 'I', 'feel', 'so', 'upset', ',', 'we', 'need', 'four', 'more', 'years', 'of', 'President', 'Obama', '1000', '%', 'certtain'] | sentiment : Negative
['For', '$', '10', 'I', "'d", 'prank', 'call', 'Obama'] | sentiment : Negative
['RT', '@Notintheface1', ':', 'Apparently', ',', 'Romney', "'s", 'entire', 'campaign', 'strategy', 'for', 'Obama', 'consists', 'of', '"', 'I', 'Know', 'You', 'Are', 'But', 'What', 'Am', 'I', '?', '"'] | sentiment : Negative
['RT', '@markknoller', ':', 'AT', 'a', 'campaign', 'fundraiser', 'tonight', ',', 'Pres', '.', 'Obama', 'again', 'denounced', 'Mitt', 'Romney', 'by', 'name', 'for', 'his', 'support', 'of', 'the', 'House', '-', 'passed', 'GOP', 'Budget', '.'] | sentiment : Positive
['@HesDanTheMan', '@edshow', 'nothing', 'will', 'come', 'of', 'it', '.', ' ', 'Had', 'it', 'been', '#', 'obama', ',', 'the', '#', 'gop', 'would', 'impeach', 'him', '.', '#', 'p2', '#'

This is where actual assignment starts wher we have to output the hidden states at every step of the encoder and decoder for a randomly chsen sentence. 

In [ ]:
r_n=random.choice(train)
tweet=r_n.tweets
indexed = [tokenizer[t] for t in tweet]        
# compute no. of words        
length = [len(indexed)]
# convert to tensor                                    
tensor = torch.LongTensor(indexed).to(device)   
# reshape in form of batch, no. of words           
tensor = tensor.unsqueeze(1).T  
# convert to tensor                          
length_tensor = torch.LongTensor(length)
# Get the model prediction                  
prediction,o1,o2= model(tensor)


In [ ]:
prediction.size()

torch.Size([1, 3])

In [ ]:
o1.size(), o2.size(), len(tweet)

(torch.Size([1, 18, 100]), torch.Size([1, 18, 100]), 18)

In [ ]:
import numpy as np

In [ ]:
encoded=o1.squeeze()
decoded=o2.squeeze()

In [ ]:
encoded.size()

torch.Size([30, 100])

In [ ]:
encoded[0].shape

torch.Size([100])

In [ ]:
print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')

In [ ]:
print(tweet, len(tweet))

['If', 'you', 'can', 'find', 'a', 'better', 'Obama', 'gif', 'than', 'this', 'one', ',', 'I', "'m", 'quitting', 'Twitter', '.', 'http://t.co/cNr0Sr3c'] 18


In [ ]:

for i in range(len(tweet)):
  enc=encoded[i]
  dec=decoded[i]
  print(f'word :{tweet[i]}  \n encoded : {enc}\n decoded : {dec}')

word :If  
 encoded : tensor([-0.1219,  0.6393,  0.8072,  0.0501,  0.4938,  0.8374,  0.4365,  0.8921,
         0.1255, -0.4568, -0.2770,  0.8646,  0.3292, -0.6725, -0.6538, -0.2788,
        -0.8048,  0.1877,  0.0111,  0.0480,  0.2366, -0.6455, -0.2108,  0.1328,
        -0.9034,  0.1650, -0.2463, -0.7071, -0.0422,  0.2194, -0.2699,  0.3182,
        -0.0832, -0.2717,  0.9182, -0.0059, -0.2682, -0.0154, -0.0785, -0.1075,
        -0.0830,  0.2120,  0.3154,  0.3100,  0.2673, -0.7040, -0.4662, -0.1212,
        -0.5743, -0.1202, -0.6514, -0.5062, -0.6050,  0.3397,  0.1468,  0.3589,
        -0.1300, -0.3899,  0.0020, -0.7171,  0.6364,  0.9270, -0.4000,  0.0403,
         0.1271,  0.4519, -0.5121, -0.1553, -0.2712, -0.4041,  0.1343, -0.6136,
        -0.0233,  0.7324,  0.2686, -0.3927, -0.8470,  0.4281, -0.3928,  0.1612,
         0.1470, -0.3926, -0.9191,  0.0896,  0.4663, -0.3839, -0.4881, -0.3946,
         0.6254, -0.8126,  0.3463,  0.6129,  0.5195, -0.4110, -0.8558, -0.8201,
         0.1622,  

In [ ]:
classify_tweet(tweet)

'Positive'